# Imports

In [2]:
import pandas as pd

In [5]:
videos_df = pd.read_csv("./data/videos.csv")
videos_df.head()

,vid,viewCount,channelId,likeCount,favoriteCount,commentCount,madeForKids,duration,publishedAt,caption,tags,thumbnails
0,hD1YtmKXNb4,40457609.0,UCX6OQ3DkcsbYNE6H8uQQuVA,2381956.0,0,102084.0,False,PT13M59S,2022-04-09T20:00:00Z,False,[],{'url': 'https://i.ytimg.com/vi/hD1YtmKXNb4/de...
1,LnlKwzc_TNA,50963596.0,UCX6OQ3DkcsbYNE6H8uQQuVA,2258114.0,0,92467.0,False,PT9M,2022-03-19T20:02:32Z,False,[],{'url': 'https://i.ytimg.com/vi/LnlKwzc_TNA/de...
2,3jS_yEK8qVI,84247720.0,UCX6OQ3DkcsbYNE6H8uQQuVA,3257710.0,0,137574.0,False,PT8M2S,2022-01-29T21:00:01Z,False,[],{'url': 'https://i.ytimg.com/vi/3jS_yEK8qVI/de...
3,00NgUctWoLQ,62739606.0,UCX6OQ3DkcsbYNE6H8uQQuVA,2985456.0,0,135606.0,False,PT12M9S,2021-12-18T21:00:00Z,True,[],{'url': 'https://i.ytimg.com/vi/00NgUctWoLQ/de...
4,x9TQ6culXIA,50799082.0,UCX6OQ3DkcsbYNE6H8uQQuVA,2682421.0,0,91080.0,False,PT12M5S,2021-12-16T21:00:00Z,True,[],{'url': 'https://i.ytimg.com/vi/x9TQ6culXIA/de...


In [6]:
channels_df = pd.read_csv("./data/channels.csv")
channels_df.head()

,channel_name,cid,videoIds,videoCount,totalViews,totalVideos,subscriberCount
0,MrBeast,UCX6OQ3DkcsbYNE6H8uQQuVA,"['hD1YtmKXNb4', 'LnlKwzc_TNA', '3jS_yEK8qVI', ...",87,15464704561,721,94200000
1,Bearhug,UCOqODGR-AoOTkxB74sc1Xyw,"['WykBVtROETs', 'pTO91Wz41Qg', '_5ezsJAVNG4', ...",85,945208397,421,3750000
2,GoodDayOfficial,UC8XqGKvkVgmL_MCs4irRKqw,"['kGzvNcNxC7k', 'Q5MQrclGYCE', '2xGaup5tTGM', ...",95,1323947973,2060,3440000
3,Graham Stephan,UCV6KDgJskWaEckne5aPA0aQ,"['wTgHS0WA9Ys', 'mu8Sm7UXOHo', '8RTmvUZlSSk', ...",100,371492787,768,3770000
4,spin9,UCovADuA7KBuMFORurTzL86A,"['-cFaMAb07xE', 'IzP_Fq0HR3k', 'IBLdRiPcC0Y', ...",100,127864609,698,638000
